# Slab projections of constrained 2DTM results

Here, the 60S large ribosomal subunit locations and orientations are used to constrain the 2DTM search for the 40S small ribosomal subunit. 
We reuse some code for slab generation to visualize the number of 40S subunits identified in isolation compared with the 40S subunits found using the constrained search.

In [ ]:
import numpy as np
import pandas as pd
import mrcfile
import torch
import matplotlib.pyplot as plt
from scipy.ndimage import rotate
from scipy.ndimage import map_coordinates
from scipy.spatial.transform import Rotation

# import napari
import torch_fourier_rescale

## Downloading, importing, and loading data

### Load in the simulated templates

Each of these volumes have been simulated and used to search a micrograph using 2DTM.
We load the MRC files into numpy arrays and additionally find their simulated pixel sizes.
These data will be used to downsample the volumes before placing them in the slab volumes.

In [2]:
template_60S_path = "/data/papers/Leopard-EM_paper_data/maps2/60S_map_px0.936_bscale0.5.mrc"
template_40S_path = "/data/papers/Leopard-EM_paper_data/maps2/SSU-body_map_px0.936_bscale0.5.mrc"

In [3]:
# Load template files and print their shapes
with mrcfile.open(template_60S_path, mode="r") as f:
    template_60S = f.data.copy()
    px_60S = f.voxel_size.x.item()
    print(f"60S template shape: {template_60S.shape}")
    print(f"60S template voxel shape: {px_60S:.3f}")
    print()

with mrcfile.open(template_40S_path, mode="r") as f:
    template_40S = f.data.copy()
    px_40S = f.voxel_size.x.item()
    print(f"40S template shape: {template_40S.shape}")
    print(f"40S template voxel shape: {px_40S:.3f}")
    print()

60S template shape: (512, 512, 512)
60S template voxel shape: 0.936

40S template shape: (512, 512, 512)
40S template voxel shape: 0.936



### Load in the full micrograph

In [4]:
img_path = "/data/papers/Leopard-EM_paper_data/xe30kv/all_mgraphs/xenon_252_000_0.0_DWS.mrc"


with mrcfile.open(img_path, mode="r") as f:
    img = f.data.copy()
    print(f"60S micrograph shape: {img.shape}")
    print()

60S micrograph shape: (4096, 4096)



### Match template results for individual 40S and 60S searches

These are csv files parsed into a DataFrame which contains all necessary information about each located particle.

*Note: we apply an additional filtering step to exclude peaks within 10 pixels to avoid selecting the same particle multiple times.*

In [5]:
results_60S_df_path =   "/data/papers/Leopard-EM_paper_data/xe30kv/results_refine_tm_60S_noB/xenon_235_000_0.0_DWS_refined_results.csv"
results_40S_df_path = "/data/papers/Leopard-EM_paper_data/xe30kv/results_match_tm_40S-body_noB/xenon_235_000_0.0_DWS_results.csv"

In [6]:
results_60S_df = pd.read_csv(results_60S_df_path)
results_40S_df = pd.read_csv(results_40S_df_path)
len(results_60S_df), len(results_40S_df)

(142, 12)

### Constrained orientation search results for 40S

Again, this is a csv file parsed into a DataFrame with similar content, but this time the data has just been processed using the constrained search program.

In [7]:
constrained_40S_df_path = "/data/papers/Leopard-EM_paper_data/xe30kv/results_all_steps_5/step_4/xenon_235_000_0.0_DWS_constrained_results_above_threshold.csv"


In [8]:
constrained_40S_df = pd.read_csv(constrained_40S_df_path)
constrained_40S_df.shape

(86, 53)

In [9]:
# replace the 'refined_relative_defocus' column with that in the results_60S_df
#
# The rationale behind this is because the two models were aligned together, that
# their relative z-positions should be the same. However, the 40S particle will
# *physically* have a different defocus value than the 60S particle in the sample, thus
# we should account for this during CTF computation since projections only happen in the
# x-y plane.
#
# The 'particle_index' column maps 40S particles to 60S particles, so we can use it to
# replace the 'refined_relative_defocus' column in the constrained_40S_df with the
# corresponding values from the results_60S_df.
for i, row in constrained_40S_df.iterrows():
    particle_index = row["particle_index"]
    row_60S = results_60S_df[results_60S_df["particle_index"] == particle_index]
    if row_60S.shape[0] == 1:
        real_df = row_60S["refined_relative_defocus"].values[0]
        constrained_40S_df.at[i, "refined_relative_defocus"] = real_df
    else:
        raise ValueError(
            f"Expected one row for particle index {particle_index}, but got {row_60S.shape[0]} rows."
        )
constrained_40S_df.refined_relative_defocus

0     100.0
1     -60.0
2     -40.0
3     160.0
4    -220.0
      ...  
81   -100.0
82    -80.0
83   -340.0
84   -300.0
85   -400.0
Name: refined_relative_defocus, Length: 86, dtype: float64

In [10]:
def filter_close_results(df: pd.DataFrame, threshold: float = 10.0) -> pd.DataFrame:
    """Filters rows in a DataFrame based on proximity of 'pos_x' and 'pos_y' values.

    NOTE: This function give preference to rows with a higher 'scaled_mip' value
    when filtering out close results.

    Parameters:
    -----------
    df : pd.DataFrame
        DataFrame containing 'pos_x' and 'pos_y' columns.
    threshold : float
        Distance threshold for filtering. Rows with 'pos_x' and 'pos_y' values
        within this distance of each other will be filtered out.

    Returns:
    --------
    pd.DataFrame
        Filtered DataFrame containing only rows that are not close to each other
        based on the specified threshold.
    """
    filtered_df = df.copy()
    filtered_df = filtered_df.sort_values(by="scaled_mip", ascending=False)
    filtered_df = filtered_df.reset_index(drop=True)

    # List to store indices of rows to keep
    indices_to_keep = []

    for i, row in filtered_df.iterrows():
        # Check if this row is close to any already kept row
        is_close = False
        for kept_idx in indices_to_keep:
            kept_row = filtered_df.loc[kept_idx]
            if (
                abs(row["pos_x"] - kept_row["pos_x"]) < threshold
                and abs(row["pos_y"] - kept_row["pos_y"]) < threshold
            ):
                is_close = True
                break

        # If not close to any kept row, keep this one
        if not is_close:
            indices_to_keep.append(i)

    # Return filtered dataframe with only the kept indices
    return filtered_df.loc[indices_to_keep].sort_index()


results_60S_df = filter_close_results(results_60S_df, threshold=10)
results_40S_df = filter_close_results(results_40S_df, threshold=10)
constrained_40S_df = filter_close_results(constrained_40S_df, threshold=10)

## Helper functions for slab creation and visualization

### Creating slab with translations and rotations

Here, the slab is created to be the same size as the original micrograph if it were down-sampled.
This means the slab can be directly overlaid on the micrograph to visualize structure position

In [11]:
def create_empty_slab(
    image_shape: tuple[int, int],
    image_pixel_size: float,  # Angstroms
    slab_pixel_size: float,  # Angstroms
    slab_thickness: float,  # Angstroms
) -> np.ndarray:
    """Create an empty slab with the specified pixel size and thickness.

    Parameters
    ----------
    image_shape : tuple[int, int]
        Shape of the image (height, width).
    image_pixel_size : float
        Pixel size of the image, in Angstroms.
    slab_pixel_size : float
        Desired voxel size of the slab, in Angstroms (isotropic).
    slab_thickness : float
        Thickness of the slab, in Angstroms.
    """
    # Slab spans the same physical area as the image, but will have a different
    # voxel pitch.
    height = image_shape[0] * image_pixel_size
    width = image_shape[1] * image_pixel_size

    slab_height = int(height / slab_pixel_size)
    slab_width = int(width / slab_pixel_size)
    slab_depth = int(slab_thickness / slab_pixel_size)

    slab = np.zeros((slab_height, slab_width, slab_depth), dtype=np.float32)

    return slab

### Helper function to place a smaller volume into a larger volume

Note that this operates on integer coordinates (no sub-voxel accuracy), nor does this handle complex bounds checking.
It will just raise an error if the slab bounds are exceeded.

Also note that the rotation format is 'xyz' rather than the typical 'ZYZ' Euler angle format used by Leopard-EM

In [12]:
def rotate_array(array, orient):
    phi = orient[0]
    the = orient[1]
    psi = orient[2]

    # create meshgrid
    dim = array.shape
    ax = np.arange(dim[0])
    ay = np.arange(dim[1])
    az = np.arange(dim[2])
    coords = np.meshgrid(ax, ay, az)

    # stack the meshgrid to position vectors, center them around 0 by substracting dim/2
    xyz = np.vstack(
        [
            coords[0].reshape(-1) - float(dim[0]) / 2,  # x coordinate, centered
            coords[1].reshape(-1) - float(dim[1]) / 2,  # y coordinate, centered
            coords[2].reshape(-1) - float(dim[2]) / 2,
        ]
    )  # z coordinate, centered

    # create transformation matrix
    r = Rotation.from_euler("ZYZ", [phi, the, psi], degrees=True)
    mat = r.as_matrix()

    # apply transformation
    transformed_xyz = np.dot(mat, xyz)

    # extract coordinates
    x = transformed_xyz[0, :] + float(dim[0]) / 2
    y = transformed_xyz[1, :] + float(dim[1]) / 2
    z = transformed_xyz[2, :] + float(dim[2]) / 2

    x = x.reshape((dim[1], dim[0], dim[2]))
    y = y.reshape((dim[1], dim[0], dim[2]))
    z = z.reshape(
        (dim[1], dim[0], dim[2])
    )  # reason for strange ordering: see next line

    # the coordinate system seems to be strange, it has to be ordered like this
    new_xyz = [y, x, z]

    # sample
    arrayR = map_coordinates(array, new_xyz, order=1)

    return arrayR


def place_into_larger_volume(
    small_volume: np.ndarray,
    large_volume: np.ndarray,
    position: tuple[int, int, int],
    orientation: tuple[float, float, float],
) -> np.ndarray:
    """Transform a small volume and place it into a larger volume.

    NOTE: The orientations are in ZYZ format, volumes are in ZYX format

    Parameters
    ----------
    small_volume : np.ndarray
        The small volume to be placed, shape (depth, height, width) in ZYX format.
    large_volume : np.ndarray
        The larger volume into which the small volume will be placed, shape
        (depth, height, width) in ZYX format.
    position : tuple[int, int, int]
        The (z, y, x) position in the larger volume where the small volume will be placed.
    orientation : tuple[float, float, float]
        The rotation angles (in degrees) for ZYZ Euler angles.

    Returns
    -------
    np.ndarray
        The larger volume with the small volume placed and rotated inside it.
    """
    # Calculate the position in the larger volume (only integer coordinates)
    z, y, x = position
    z_end = z + small_volume.shape[0]
    y_end = y + small_volume.shape[1]
    x_end = x + small_volume.shape[2]

    # Check if the small volume fits into the larger volume
    if (
        z < 0
        or y < 0
        or x < 0
        or z_end > large_volume.shape[0]
        or y_end > large_volume.shape[1]
        or x_end > large_volume.shape[2]
    ):
        print(f"Position: {position}")
        print(f"Small volume shape: {small_volume.shape}")
        print(f"Larger volume shape: {large_volume.shape}")
        raise ValueError("Volume is out of bounds!")

    volume_rotated = rotate_array(small_volume.clone(), orientation)

    # Place the rotated small volume into the larger volume
    large_volume[z:z_end, y:y_end, x:x_end] += volume_rotated

    return large_volume

### Helper function to render volume from results DataFrame

In [13]:
def construct_slab_from_results_df(
    template_volume: np.ndarray,
    results_df: pd.DataFrame,
    image_shape: tuple[int, int],
    image_pixel_size: float,
    slab_pixel_size: float,
    slab_z_min: float = None,
    slab_z_max: float = None,
    use_refined_columns: bool = True,
) -> np.ndarray:
    """Takes in a results_df DataFrame and constructs a slab for the volume.

    Parameters
    ----------
    template_volume : np.ndarray
        The template volume to be placed into the slab, shape (depth, height, width).
    results_df : pd.DataFrame
        DataFrame containing the results with columns 'pos_x', 'pos_y', 'relative_defocus',
        'phi', 'theta', and 'psi'.
    image_shape : tuple[int, int]
        Shape of the image (height, width) that the slab will cover.
    image_pixel_size : float
        Pixel size of the image, in Angstroms.
    slab_pixel_size : float
        Desired voxel size of the slab, in Angstroms (isotropic).
    slab_z_min : float, optional
        Minimum Z position of the slab in Angstroms. If None, it is calculated
        from the results_df.
    slab_z_max : float, optional
        Maximum Z position of the slab in Angstroms. If None, it is calculated
        from the results_df.
    use_refined_columns : bool, optional
        If True, then preference is given to refined columns in the results_df.
        If False, the original columns are used. Falls back to original columns
        if the refined columns are not present.

    Returns
    -------
    np.ndarray
        The constructed slab containing the placed template volumes, shape
        (slab_height, slab_width, slab_depth).
    """
    pos_x_col = "pos_x"
    pos_y_col = "pos_y"
    phi_col = "phi"
    theta_col = "theta"
    psi_col = "psi"
    relative_defocus_col = "relative_defocus"
    if use_refined_columns:
        if (
            "refined_pos_x" in results_df.columns
            and "refined_pos_y" in results_df.columns
            and "refined_phi" in results_df.columns
            and "refined_theta" in results_df.columns
            and "refined_psi" in results_df.columns
            and "refined_relative_defocus" in results_df.columns
        ):
            pos_x_col = "refined_pos_x"
            pos_y_col = "refined_pos_y"
            phi_col = "refined_phi"
            theta_col = "refined_theta"
            psi_col = "refined_psi"
            relative_defocus_col = "refined_relative_defocus"
        else:
            print("Refined columns not found, using original columns.")

    # Find minimum/maximum defocus values to set zero position
    min_defocus = results_df[relative_defocus_col].min()
    max_defocus = results_df[relative_defocus_col].max()

    # Use defocus range as slab thickness if not provided
    if slab_z_min is None:
        slab_z_min = min_defocus
    if slab_z_max is None:
        slab_z_max = max_defocus

    slab_thickness = slab_z_max - slab_z_min 
    slab_thickness += (template_volume.shape[0] + 1) * slab_pixel_size

    slab = create_empty_slab(
        image_shape=image_shape,
        image_pixel_size=image_pixel_size,
        slab_pixel_size=slab_pixel_size,
        slab_thickness=slab_thickness,
    )

    for i, row in results_df.iterrows():
        print(f"Placing volume {i + 1} of {len(results_df)}")
        # Get full image position and convert to slab coordinates
        position = [
            row[pos_x_col] * image_pixel_size,
            row[pos_y_col] * image_pixel_size,
            float(row[relative_defocus_col]) - slab_z_min,
        ]
        position = np.array(position)
        position = np.round(position / slab_pixel_size).astype(int)
        position = tuple(position.tolist())

        orientation = [row[phi_col], row[theta_col], row[psi_col]]

        slab = place_into_larger_volume(
            small_volume=template_volume,
            large_volume=slab,
            position=position,
            orientation=orientation,
        )

    # # Slab is in XYZ format, but MRC are generally defined in ZYX format.
    # slab = np.transpose(slab, (2, 0, 1))  # Change to ZYX format

    return slab

## Generating slabs for visualization


In [14]:
template_60S_rescaled, px_60S_rescaled = torch_fourier_rescale.fourier_rescale_3d(
    image=torch.from_numpy(template_60S),
    source_spacing=px_60S,
    target_spacing=5.0,
)
px_60S_rescaled = px_60S_rescaled[0].item()

print(f"Rescaled 60S template voxel shape: {px_60S_rescaled:.3f} Angstroms")

Rescaled 60S template voxel shape: 4.992 Angstroms


In [15]:
template_40S_rescaled, px_40S_rescaled = torch_fourier_rescale.fourier_rescale_3d(
    image=torch.from_numpy(template_40S),
    source_spacing=px_40S,
    target_spacing=5.0,
)
px_40S_rescaled = px_40S_rescaled[0].item()

print(f"Rescaled 40S template voxel shape: {px_40S_rescaled:.3f} Angstroms")

Rescaled 40S template voxel shape: 4.992 Angstroms


### Finding minimum/maximum relative defocus for each dataframe

Since we want the z depth of particles to be consistent across all generated slabs, we look at the minimum and maximum relative defocus columns in each of the DataFrames taking the most extreme values.

In [16]:
slab_z_min = min(
    results_60S_df["refined_relative_defocus"].min(),
    results_40S_df["relative_defocus"].min(),
    constrained_40S_df["refined_relative_defocus"].min(),
).item()
slab_z_max = max(
    results_60S_df["refined_relative_defocus"].max(),
    results_40S_df["relative_defocus"].max(),
    constrained_40S_df["refined_relative_defocus"].max(),
).item()
slab_z_min, slab_z_max

(-980.0, 760.0)

In [17]:
slab_60S = construct_slab_from_results_df(
    template_volume=template_60S_rescaled,
    results_df=results_60S_df,
    image_shape=img.shape,
    image_pixel_size=px_60S,
    slab_pixel_size=px_60S_rescaled,
    slab_z_max=slab_z_max,
    slab_z_min=slab_z_min,
)
slab_60S.shape

Placing volume 1 of 113


NameError: name 'map_coordinates' is not defined

In [18]:
# Save the slab to a MRC file
slab_60S_mrc_path = "slab_60S_constrained.mrc"
with mrcfile.new(slab_60S_mrc_path, overwrite=True) as mrc:
    mrc.set_data(slab_60S.astype(np.float32))
    mrc.voxel_size = (px_60S_rescaled, px_60S_rescaled, px_60S_rescaled)
    mrc.update_header_from_data()
    print(f"Slab saved to {slab_60S_mrc_path}")

NameError: name 'slab_60S' is not defined

In [19]:
slab_40S = construct_slab_from_results_df(
    template_volume=template_40S_rescaled,
    results_df=results_40S_df,
    image_shape=img.shape,
    image_pixel_size=px_40S,
    slab_pixel_size=px_40S_rescaled,
    slab_z_max=slab_z_max,
    slab_z_min=slab_z_min,
)
slab_40S.shape

Refined columns not found, using original columns.
Placing volume 1 of 11


NameError: name 'map_coordinates' is not defined

In [20]:
# Save the slab to a MRC file
slab_40S_mrc_path = "slab_40S_unconstrained.mrc"
with mrcfile.new(slab_40S_mrc_path, overwrite=True) as mrc:
    mrc.set_data(slab_40S.astype(np.float32))
    mrc.voxel_size = (px_40S_rescaled, px_40S_rescaled, px_40S_rescaled)
    mrc.update_header_from_data()
    print(f"Slab saved to {slab_40S_mrc_path}")

NameError: name 'slab_40S' is not defined

In [21]:
slab_40S_constrained = construct_slab_from_results_df(
    template_volume=template_40S_rescaled,
    results_df=constrained_40S_df,
    image_shape=img.shape,
    image_pixel_size=px_40S,
    slab_pixel_size=px_40S_rescaled,
    slab_z_max=slab_z_max,
    slab_z_min=slab_z_min,
)
slab_40S_constrained.shape

Placing volume 1 of 76
Placing volume 2 of 76
Placing volume 3 of 76
Placing volume 4 of 76
Placing volume 5 of 76
Placing volume 6 of 76
Placing volume 7 of 76
Placing volume 8 of 76
Placing volume 9 of 76
Placing volume 10 of 76
Placing volume 11 of 76
Placing volume 12 of 76
Placing volume 13 of 76
Placing volume 14 of 76
Placing volume 15 of 76
Placing volume 16 of 76
Placing volume 17 of 76
Placing volume 18 of 76
Placing volume 19 of 76
Placing volume 20 of 76
Placing volume 21 of 76
Placing volume 22 of 76
Placing volume 23 of 76
Placing volume 24 of 76
Placing volume 25 of 76
Placing volume 26 of 76
Placing volume 27 of 76
Placing volume 28 of 76
Placing volume 29 of 76
Placing volume 30 of 76
Placing volume 31 of 76
Placing volume 32 of 76
Placing volume 33 of 76
Placing volume 34 of 76
Placing volume 35 of 76
Placing volume 36 of 76
Placing volume 37 of 76
Placing volume 38 of 76
Placing volume 39 of 76
Placing volume 40 of 76
Placing volume 41 of 76
Placing volume 42 of 76
P

(767, 767, 441)

In [22]:
# Save the slab to a MRC file
slab_40S_constrained_path = "slab_40S_constrained.mrc"
with mrcfile.new(slab_40S_constrained_path, overwrite=True) as mrc:
    mrc.set_data(slab_40S_constrained.astype(np.float32))
    mrc.voxel_size = (px_40S_rescaled, px_40S_rescaled, px_40S_rescaled)
    mrc.update_header_from_data()
    print(f"Slab saved to {slab_40S_constrained_path}")

Slab saved to slab_40S_constrained.mrc
